# Buchhandelsdaten in Vufind als Grundlage für PDA (Patron Driven Aquisition) am MPIfG / Version 2: Updates laden statt Vollimport

Einbindung von freundlicherweise von Schweitzer Fachinformation zur Verfügung gestellten Daten.   
(Auswahl über passend konfigurierte Neuerscheinungsabfragen in unserem Kundenprofil).   

#### Gründe für die Umstellung auf Updates: Durch Einspielen der Daten in Vufind ergibt sich eine zeitliche Diskrepanz, in der Titel bereits in Aleph gelöscht sind, aber in Vufind noch bestellbar. Zudem werden die Aleph-IDs hochgezähl, denn bei jedem Einspielen werden neue Nummern vergeben. Durch die Updates bleiben die Titel weiterhin verfügbar und das Hochzählen der IDs hält sich im Rahmen. Das Skript beschleunigt sich durch Reduzierung der Abfragen am Server.
*Kleiner Nachteil: der manuelle Aufwand vergrößert sich. Mehrere Dateien müssen in Aleph eingespielt und verarbeitet werden, aber hält sich nach wie vor im vertretbaren Rahmen.*

Das Jupyter Notebook arbeitet mit Python 3.8.1 und wurde mit Visual Studio Code 1.62.3 erstellt 


#### Arbeitsschritte im Code:

> Vorarbeiten:   
  - Notwendige Pandas Libraries aufrufen
  - Serverprüfung auf funktionierende Verbindung zum Aleph-X-Server    


> Daten abholen und einlesen:   
  1. Buchhandelsdaten von Schweitzer 
  2. Aleph-Konkordanz Aleph-ID /Schweitzer ID  (erzeugt tagesaktuell per p-print-03 in Aleph)
     - Aufbereiten der Daten: Schweitzer ID extrahieren und Aleph-ID mit Nullen auffüllen
  3. Daten zusammenführen in einem df    


> Daten aufbereiten:   
  1. Buchhandelsdaten prüfen und vorbereiten
     1. Identifizierung von Titeln in Aleph, die nicht mehr im Datensatz sind und schreiben in Datei "ges02_loeschen_1"
     2. Dublettencheck innerhalb der Buchhandelsdaten
     3. Trennung der Daten in "in Aleph vorhanden" und "neu"  
  2. Bestandsabfragen:
     1. Ganz neue Titel   
        - Bestandsabgleich durch Abfragen (GES und EBX) auf dem Aleph-Server   
     2. In Aleph vorhanden
        - Bestandsabgleich durch Abfrage EBX auf dem Aleph-Server  
     3. Exportvorbereitungen:
        1. Neue Titel
        2. In Aleph vorhandene Titel   
            Identifizierung von zu löschenden Titeln "ges02_loeschen_2" und der zu aktualisierenden
  3. Exportdateien aufbereiten:
     1. Aufbereiten neuer Titel
     2. Aufbereiten der vorhandenen Titel   


> Informationssammlung
   1. Log-Datei mit Rahmendaten wird fortlaufend geschrieben
   2. Ausgabe bestimmter Titelgruppen als csv-Datei  


<hr>

## Vorarbeiten

### Pandas Libraries laden

In [1]:
import pandas as pd                                                # für das Arbeiten mit der CSV-Datei
import urllib.request                                              # für das Abrufen der URL
import requests                                                    # für die Bestandsabfragen 
pd.options.mode.chained_assignment = None                          # default='warn' abschalten beim Beschreiben der neuen Spalten
import time                                                        # für das Schreiben des Datums Logdatei und Excel-Export und Arbeiten mit dem Erscheinungsdatum
import datetime                                                    # für das Berechnen des Updates
import numpy as np                                                 # für das Bearbeiten von Spalten


### Prüfung, ob die Verbindung zum Aleph-Server für Abfragen korrekt funktioniert:

    Nur zugelassene IPs können diese Schnittstelle abfragen.

In [2]:
test= "http://aleph.mpg.de/X?op=find&base=ges01&request=IBS=9783482648434"

reply = requests.get(test).text
a = reply.find("Forbidden")
b =  reply.find("?xml")

if (a > 50):
    print("Es gibt ein Problem mit dem Server")
if (b == 1):
    print("Der Server antwortet korrekt")

Der Server antwortet korrekt


In [3]:
test= "http://aleph.mpg.de/X?op=find&base=ges01&request=IBS=9783865058041"

reply = requests.get(test).text 

print(reply) 

<?xml version = "1.0" encoding = "UTF-8"?>
<find>
<set_number>921747</set_number>
<no_records>000000001</no_records>
<no_entries>000000001</no_entries>
<session-id>2Y3372BU7TGHNBB4K4RKQPHNL9DQMMVU9L38TKA8LV5TC7768F</session-id>
</find>
 


<hr>


## Datensätze abholen und einlesen

# Erste Schritte manuell durchführen, um enthaltene Prüfroutinen im Blick zu behalten

<hr>

### 1. Datensätze von Schweitzer

*einlesen in df*

In [35]:
url = "https://content.schweitzer-online.de/static/content/export/mpifg/export.csv"  # Abruf, der von Schweitzer zur Verfügung gestellten Daten
checkout_file = "./input/export.csv"  
urllib.request.urlretrieve(url, checkout_file)

('./input/export.csv', <http.client.HTTPMessage at 0x21067428790>)

In [40]:
df = pd.read_csv('./input/export.csv', encoding = 'UTF-8', sep=';' , keep_default_na=False , low_memory=False) # muss encoding angeben und Trennzeichen, NaN (= leere Werte) direkt beim Import entfernen, da sie später Probleme machen

In [42]:
#Aufgrund eines zusätzlichen Zeilenumbruchs im Datensatz gab es im Juni 2022 ein Problem, darum kleine Prüfroutine eingebaut ob in der Import-Datei die Anzahl der Datensätze mit der Anzahl der object-Ids übereinstimmt
g = df.shape[0]
h= df['object_id'].count()

print("Zeilen in Datei:", g)
print("Object_IDs:     ", h)


Zeilen in Datei: 66240
Object_IDs:      66240


# Ab dem nächsten Schritt kann Skript durchlaufen, Prüfroutinen sind erfolgt

<hr>

### 2. Aleph-Konkordanz Aleph-ID / Schweitzer ID

In [11]:
now = time.strftime('%Y%m%d')  # %H:%M:%S
input_file = './input/ids'+now      #input_file wird tagesaktuell aus Aleph gezogen und auf diesem Wege mit der entsprechenden Endung eingelesen

In [32]:
# neu einlesen der Aleph-IDs mittels Code, da sonst Object-Ids abgeschnitten wurden
with open(input_file) as f:
    data = f.readlines()

df = pd.DataFrame(data[0:]) # 0 weil ich keinen header habe

# use expand to split strings into separate columns
df_alephIDs = df[0].str.rsplit(expand=True)
# fix column names
df_alephIDs.columns = ["ids","Field","L","Content"]

df_alephIDs['LEN'] = df_alephIDs['Content'].apply(len)
print(df_alephIDs)

             ids Field  L       Content  LEN
0      000129218   020  L    $$a2258402   10
1      000129219   020  L    $$a2522807   10
2      000129220   020  L    $$a2308946   10
3      000129221   020  L    $$a2220214   10
4      000129222   020  L    $$a2479903   10
...          ...   ... ..           ...  ...
29970  000159265   020  L  $$a169951393   12
29971  000159266   020  L  $$a163496075   12
29972  000159267   020  L  $$a165662822   12
29973  000159268   020  L  $$a170244345   12
29974  000159269   020  L  $$a171634215   12

[29975 rows x 5 columns]


*Datenfelder aufbereiten: Schweitzer-ID extrahieren und Aleph-ID ins richtige Format bringen*

In [ ]:
df_alephIDs["object_id"] = df_alephIDs["Content"].astype(str).str.slice(start=3,stop=16).apply(int)   #Spalte mit object-Ids herausschneiden und wieder zur Zahl definieren

df_alephIDs["object_id"].value_counts() #Hier muss immer eins Stehen in der hinteren Spalte

169327230    1
170143037    1
166237816    1
171411066    1
165985917    1
            ..
163479759    1
169687954    1
166992790    1
158715772    1
169349122    1
Name: object_id, Length: 29975, dtype: int64

### 3. Daten zu einem Frame zusammenführen

In [43]:
# Zusammenführung erfolgt einmal mit concat - erlaubt die nicht mehr verwendeten Aleph-IDs zu identifizieren 
# und einmal mit join, dann erhalte ich nur die Titel, die auch im neuen Datensatz sind zur weiteren Verarbeitung!
df_oi= df.set_index("object_id")                          #object-ID zum Index für beide Datenframes
df_aleph_oi = df_alephIDs.set_index("object_id")
df_update_aleph = pd.concat([df_oi, df_aleph_oi], axis=1)

df_update_join = df_oi.join(df_aleph_oi)                    



<hr>
### Wenn es bei der obigen Zelle zu Fehlermeldung kommt, dann gibt es vermutlich doppelte SChweitzer-IDs in den Daten.

Das kann man beim Output der folgenden Zelle oben sehen:

    #nach Dubletten vom letzten Mal vorsichtshalber, check nach doppelten object-ids
    df_alephIDs["object_id"].value_counts()

Wenn da Nummern mit 2 vorhanden sind, dann muss oben der Code zur Dublettenbereinigung gestartet werden. Dies geschieht, in dem die folgende Zelle von Markdown zu Code geändert wird und durchlaufen. 

## Dabei wird eine weitere Datei erzeugt, die in Aleph gelöscht werden muss!!

WICHTIG: Anschließend, den Code wieder in Markdown verwandeln, da er ja nur in dem Sonderfall gebraucht wird. 

<hr>

# Code zur Bereinigung von Dubletten im in Aleph schon eingespielten Bestand 
# Einmalig aktivieren (Markdown > Code) und nutzen und dann wieder (Code > Markdown)

df_aleph_einzel = df_alephIDs.drop_duplicates("object_id", keep=False)  #Auslesen der Eintraege mit einzelner object_id
df_aleph_doppelt = df_alephIDs.groupby("object_id").filter(lambda g: (g.nunique() >1).any()) #Alle Sätze mit doppelten object_ids rausziehen
df_aleph_single = df_aleph_doppelt.sort_values(by=["object_id", "ids"], ascending =False).drop_duplicates(subset=["object_id"], keep='first') #Die Dublette, die behalten wird
df_aleph_dublette = df_aleph_doppelt.sort_values(by=["object_id", "ids"], ascending =False).drop_duplicates(subset=["object_id"], keep='last') #Die Dublette, die gelöscht wird
# In Aleph zu löschende in eine Datei schreiben aus df_aleph_dublette
with open("./output/ges02_dubl", "w", encoding="utf-8") as fa:  #durch das Encoding hier, kommen Sonderzeichen richtig rüber
    for i in df_aleph_dublette.index:
        fa.write(df_aleph_dublette["ids"][i]+'GES02'+'\n')
        
df_aleph_vorhanden = df_aleph_einzel.append(df_aleph_single) #Einzelne und ausgewählt Dubletten zusammenfuehren

df_oi= df.set_index("object_id")                          #Dann nochmal den Schritt, an dem es vorhin gehakt hat
df_aleph_oi = df_aleph_vorhanden.set_index("object_id")
df_update_aleph = pd.concat([df_oi, df_aleph_oi], axis=1)

df_update_join = df_oi.join(df_aleph_oi)   

# Noch kleine Rechnung zur Ueberprüfung, ob berechnete Dubletten und mit Skript ermittelte Dublettenzahl übereinstimmen
g = df_alephIDs.shape[0]
h= df_aleph_einzel.shape[0]
j = df_aleph_doppelt.shape[0]

print("Dubletten berechnet: ",  g-h)
print("Dubletten gezählt:   ", j)  


## Wenn alles gut ist, kann das Skript ab hier wieder mit "all Cells below" durchlaufen
<hr>

### LOG-Datei für den Prozess, zur Dokumentation des Imports und als Kontrollanzeige hier


In [45]:
x = df.shape[0]
print('Anzahl der enthaltenen Datensätze:', x)
#print('vorhandene ISBNs:', df_update["isbn_ean"].shape[0])

z = df_alephIDs["ids"].count()
y = df_update_join["ids"].count()

print('-------------------------')
print('Aleph-IDs anfangs', z)
print('Aleph-IDs nach join' , y)

timestr = time.strftime('%d.%m.%Y - %H:%M')

with open ('./log/pda_import_log.txt', 'a') as log:                                                  # Da diese Log-Datei nicht unmittelbar gebraucht wird, hier fortlaufendes Schreiben in eine Datei
    log.write('Logdatei PDA-Import vom ')
    log.write(timestr)
    log.write('\n------------------------------------------\n')
    log.write('Gelieferte Datensätze:             ' + str(x))

Anzahl der enthaltenen Datensätze: 66240
-------------------------
Aleph-IDs anfangs 29975
Aleph-IDs nach join 29772


## Daten aufbereiten

<hr>

### 1. Identifizierung von Titeln in Aleph-Dubletten, die nicht mehr im Datensatz sind

In [46]:
df_update_aleph.replace({np.nan: None})

,isbn_ean,title,subtitle,contributor,publisher,series,thesis,edition_number,edition_text,media_type,...,cover,predecessor,follower,last_modified,watchlist_name,ids,Field,L,Content,LEN
object_id,,,,,,,,,,,,,,,,,,,,,
217164,9.78341e+12,Unternehmensbewertung für Juristen,,Dirk Hachmeister;Matthias Schüppen,C.H.BECK,Neue Juristische Wochenschrift (NJW) - Praxis ...,,1,,hardcover,...,https://content.schweitzer-online.de/static/ca...,,,2025-02-03 05:50:21,Himbeere Wirtschaft #181.2,None,None,None,None,None
218910,9.78341e+12,Grundkurs Steuerrecht,,Rainer Wernsmann;Christian Thiemann,C.H.BECK,Grundkurse,,1,,hardcover,...,https://content.schweitzer-online.de/static/ca...,,,2025-02-05 06:58:17,brombeere Politik #69.1,000129231,020,L,$$a218910,9
237277,9.78383e+12,Käuferverhalten,Eine marketingorientierte Einführung,Alfred Kuß;Torsten Tomczak;Silke Lennerts,UTB,Grundwissen der Ökonomik Band 1604,,5,"5., komplett überarbeitete Auflage 2017",hardcover,...,https://content.schweitzer-online.de/static/ca...,9783825216047,,2025-03-05 20:40:44,Himbeere Wirtschaft #26,None,None,None,None,None
240369,9.78383e+12,WTO-Recht in Fällen,,Jan Neumann;Christian Pitschas,Nomos,NomosStudium,,2,,hardcover,...,https://content.schweitzer-online.de/static/ca...,9783832904241,,2025-03-05 20:41:38,Himbeere Wirtschaft #112.2,000129233,020,L,$$a240369,9
261723,9.78349e+12,Gesundheitsökonomie,Eine Einführung,Kornelia van der Beek;Gregor van der Beek;Wilf...,De Gruyter Oldenbourg,de Gruyter Studium,,,"2., überarbeitete und erweiterte Auflage",hardcover,...,https://content.schweitzer-online.de/static/ca...,9783486586862,,2025-02-03 06:13:52,Himbeere Wirtschaft #119.1,000129303,020,L,$$a261723,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173032813,9.78384e+12,Visuelle Transformationen des Exils,"Die jüdischen Künstlerinnen Grete Stern, Hedy ...",Christina Wieder,transcript,Historische Geschlechterforschung 21,,1,,hardcover,...,https://content.schweitzer-online.de/static/ca...,,,2025-03-11 19:58:50,brombeere Politik #200.2,None,None,None,None,None
173032814,9.78384e+12,Talking Politics and Society Again,Reengaging with Fellow Citizens,Hans Blokland,transcript,Kultur und soziale Praxis,,1,,hardcover,...,https://content.schweitzer-online.de/static/ca...,,,2025-03-11 19:58:50,brombeere Politik #200.2,None,None,None,None,None
173052865,9.78234e+12,"Europe, as-tu une âme ?",Essai pour une pédagogie de l'Europe,Vincent Carbonel,Editions L'Harmattan,,,,,hardcover,...,https://content.schweitzer-online.de/static/ca...,,,2025-03-05 18:40:16,brombeere Politik #200.1,None,None,None,None,None


In [47]:
df_aleph_loeschen = df_update_aleph[df_update_aleph['isbn_ean'].isna()]  


In [48]:
df_aleph_loeschen

,isbn_ean,title,subtitle,contributor,publisher,series,thesis,edition_number,edition_text,media_type,...,cover,predecessor,follower,last_modified,watchlist_name,ids,Field,L,Content,LEN
object_id,,,,,,,,,,,,,,,,,,,,,
2254200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,000129227,020,L,$$a2254200,10.0
2368410,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,000129249,020,L,$$a2368410,10.0
2430847,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,000129244,020,L,$$a2430847,10.0
2481913,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,000129235,020,L,$$a2481913,10.0
2576208,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,000129267,020,L,$$a2576208,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171787741,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,000155532,020,L,$$a171787741,12.0
171796455,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,000152210,020,L,$$a171796455,12.0
171868889,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,000152355,020,L,$$a171868889,12.0


In [49]:
# Schreiben der Loeschen-datei!

with open("./output/ges02_weg", "w", encoding="utf-8") as fa:  #durch das Encoding hier, kommen Sonderzeichen richtig rüber
    for i in df_aleph_loeschen.index:
        fa.write(df_aleph_loeschen["ids"][i]+'GES02'+'\n')

<hr>

## 2. Dublettencheck innerhalb gelieferten Buchhandelsdaten 

Aufgaben im Rahmen des Dublettencheck:
1. Dublettenkontrolle anhand von Titel, Untertitel und Autor 
   - Zunächst Behebung der unsauberen Titel / Untertitel-Trennung für korrekteren Abgleich
   - Trennung der Datensätze in Dubletten und "Einzeltitel"
     - Einzeltitel werden direkt für Bestandsprüfung vorgemerkt
     - Dubletten werden auf neueste Version reduziert und diese der Bestandsprüfungsdatei hinzugefügt

### Die Dublettenkontrolle Anhand von Titel, Untertitel und Autor

*Entfernen von Untertiteln aus der Titelspalte, Extrahieren von Untertiteln und Abgleich mit Untertitelspalte und Schreiben der vorhandenen Informationen in neue Untertitel-Spalte.   
Durch diese Spalte werden ca. 1/3 mehr Dubletten erkannt, als ohne die Bereinigung. *

In [50]:
df = df_update_join[df_update_join['isbn_ean'].notna()]  #jetzt alle die zu verarbeiten sind rausziehen
df.reset_index(inplace=True) 

In [51]:
neu = df["title"].str.split(':', n = 1, expand = True)  #Titel am 1. Doppelpunkt splitten und getrennt in neue Felder schreiben
df["title_sep"]= neu[0]
df["subtitle_sep"]= neu[1]

df["subtitle_sep"] = df["subtitle_sep"].replace(np.nan, '', regex=True) #NaN-Werte stören, darum raus damit ...

In [52]:
comparison = np.where(df["subtitle"] == df["subtitle_sep"], '', df["subtitle"])    # Abgleich - wenn in beiden das Gleiche steht, dann ursprüngliches "Subtitle"-Feld nehmen
df["subtitle_comparison"] = comparison  

comparison2 = np.where(df["subtitle"] < df["subtitle_sep"], df["subtitle_sep"], '') # Wenn nur in "subtitle_sep" Infos stehen, diese übernehmen, das ist noch nicht ganz sauber, da hier manchmal anderes steht als in "subtitle"
df["subtitle_comparison2"] = comparison2 

df["subtitle_all"] = df["subtitle_comparison"]+df["subtitle_comparison2"]          # Beide Informationen in neuer Subtitle-Spalte zusammenführen

In [53]:
df["short_title"] = df["title_sep"] + ' ' + df["subtitle_all"] + ' / ' + df["contributor"]  # aus den bereinigten Daten einen Kurztitel erzeugen, der dann für den Dublettencheck verwendet wird

In [55]:
df_dubletten = df.groupby("short_title").filter(lambda g: (g.nunique() >1).any()) # schreibt alle mehrfach vorhandenen Titel in ein eigenes Datenframe

In [56]:

df_dubl_einspielen = df_dubletten.sort_values(by=["short_title", "publication_year"], ascending =False).drop_duplicates(subset=["short_title"], keep='first')   # sortiert Dubletten nach Jahr und schreibt den jeweils ersten (= neuesten) Eintrag in neues Dataframe


In [57]:
df_ohne_dubletten = df.drop_duplicates("short_title", keep=False)       #durch ""keep=False" werden alle nicht-Dubletten rausgezogen

In [58]:
df_dubl_nicht_einspielen = df_dubletten.sort_values(by=["short_title", "publication_year"], ascending =False).drop_duplicates(subset=["short_title"], keep=False)

#Kontrolle, ob es in Dubletten Titel gibt, die schon in Aleph sind
df_dubl_nicht_einspielen["ids"].count()


0

In [59]:
df_einspielen = df_ohne_dubletten.append(df_dubl_einspielen)                    # die ausgewählten Dubletten und alle Nicht-Dubletten werden in ein Datenframe zusammengeführt
df_einspielen.reset_index(inplace=True)                                         # für weitere Bearbeitung index-reset nötig

<hr>

## 3. Überblick zu den Daten und Trennen in "in Aleph vorhanden" und "neu"

In [60]:
m = df_einspielen.shape[0]
n = df_einspielen["ids"].count()
print("Zum Einspielen:", m)
print("Davon in Aleph:", n)
print("Neue Titel:", m-n)

Zum Einspielen: 53951
Davon in Aleph: 27684
Neue Titel: 26267


In [61]:
df_in_aleph = df_einspielen.dropna(subset=['ids'])
df_in_aleph.reset_index(inplace=True)
df_in_aleph.shape[0]

# zur Prüfung am GES und Ebx Bestand geprüft werden

27684

In [62]:
df_ganz_neu = df_einspielen[df_einspielen['ids'].isnull()]

df_ganz_neu.reset_index(inplace=True)
df_ganz_neu.shape[0]

# diese müssen ebenfalls am Ges und Ebx Bestand geprüft werden, durch Trennung der Sets schnellere Bearbeitung

26267

<hr>

## 2. Bestandsabfragen



*URLs für die Abfrage über den X-Server unseres Bibliothekssystems werden erzeugt und über die ISBN eine Abfrage auf Bestand gemacht. Die Abfrage funktioniert nur für zugelassene IPs (darum oben die Prüfung).  
Für die Abfrage in unseren Bestand ist die ISBN sehr gut, da in den Titeldaten alle im Buch befindlichen ISBNs - auch die anderer Ausgabeformen - mit übernommen sind. Beim MPG-Ebooks Katalog handelt sich um Daten von Verlagen, die sich in ihrer Qualität und Informationsumfang sehr unterscheiden. Hier wird noch zu prüfen sein, inwieweit ein anderer Abfragemechansimus gewählt werden muss.*  

### 1. Ganz neue Titel  

= df_ganz_neu


In [29]:
df_ganz_neu["url_ges"] = df_ganz_neu["isbn_ean"].apply(lambda x: f"ges_link{x}".replace('ges_link','http://aleph.mpg.de/X?op=find&base=ges01&request=IBS='))

#Es funktionierte nicht, dass die URLs an die vorhandenen ISBNs einfach so angefügt werden, darum der Workaround mit einem Platzhalter, der sich dann über replace vom richtigen Link überschreiben ließ.

#### URLs für Ebooks-Katalog erzeugen 

In [30]:
df_ganz_neu["url_ebx"] = df_ganz_neu["isbn_ean"].apply(lambda x: f"ebx_link{x}".replace('ebx_link', 'http://aleph.mpg.de/X?op=find&base=ebx01&request=IBN='))


#### Abfragen beim Server

##### Zunächst für die Daten des MPIfG 

*Vorgehensweise: Abfrage und Sammeln der Antworten in einer Datei, diese Antworten werden dann in Ausdrücke "übersetzt" - "vorhanden" und "neu" und diese Daten in eine Spalte ins Dataframe zur weiteren Auswertung übertragen.   
Schwierigkeit hier war, die Sammlung der Antworten zu den einzelnen Titeln, um sie in das Datenframe einzuspielen. Der störende XML-Header der Antworten wird erst gar nicht in die Datei geschrieben.   *   

In [31]:
with open('./input/server_responses', 'w') as fn:  
    for url in df_ganz_neu["url_ges"]:
        reply = requests.get(url).text
        a = reply.replace('<?xml version = "1.0" encoding = "UTF-8"?>', '') 
        fn.write(a)

with open('./input/server_responses', 'r') as f:
    with open('./input/server_responses_transfered', 'w') as fr:
        for line in f:
            if 'empty' in line:
                fr.write('neu\n')
            elif 'no_records' in line:
                fr.write('vorhanden\n')

df_fwf = pd.read_fwf('./input/server_responses_transfered', names=["Abfrage_ges"])
df_result = df_ganz_neu.join(df_fwf)                    #df_result = pd.concat([df_ganz_neu, df_fwf], axis=1), funktioniert nicht mehr                  

    Randnotiz: 
    Bei 2400 Titels brauchte der Abgleich ca 350 Sekunden

In [32]:
#Kontrollabfrage, ob für alle Titel auch Treffer da sind, wird mittelfristig rausfallen
x = df_ganz_neu.shape[0]
y = df_fwf.shape[0]
print('Anzahl der eingelesenen Datensätze:', x, '\nAnzahl der Antworten vom Server:   ', y)

Anzahl der eingelesenen Datensätze: 5329 
Anzahl der Antworten vom Server:    5329


##### Datenabgleich mit dem Bestand des MPG Ebooks-Katalog

*Vorgehensweise analog Bestandsabfrage MPI.*

In [33]:
with open('./input/server_responses_ebx', 'w') as fn:  
    for url in df_result["url_ebx"]:
        reply = requests.get(url).text
        a = reply.replace('<?xml version = "1.0" encoding = "UTF-8"?>', '') 
        fn.write(a)

with open('./input/server_responses_ebx', 'r') as f:
    with open('./input/server_responses_transfered_ebx', 'w') as fr:
        for line in f:
            if 'empty' in line:
                fr.write('neu\n')
            elif 'no_records' in line:
                fr.write('vorhanden\n')

df_fwf_ebx = pd.read_fwf('./input/server_responses_transfered_ebx', names=["Abfrage_ebx"])
df_result_neu = df_result.join(df_fwf_ebx)           #df_result_neu = pd.concat([df_result, df_fwf_ebx], axis=1)

In [34]:
#Kontrollabfrage, ob für alle Titel auch Treffer da sind
x = df_result_neu.shape[0]
y = df_fwf_ebx.shape[0]
print('Anzahl der eingelesenen Datensätze:', x, '\nAnzahl der Antworten vom Server:   ', y)

Anzahl der eingelesenen Datensätze: 5329 
Anzahl der Antworten vom Server:    5329


<hr>

### 2. Titel in Aleph vorhanden

= df_in_aleph

*Hier reicht Abgleich mit Ebooks, das erworbene Bücher manuell gelöscht werden.*

In [35]:
df_in_aleph["url_ges"] = df_in_aleph["isbn_ean"].apply(lambda x: f"ges_link{x}".replace('ges_link', 'http://aleph.mpg.de/X?op=find&base=ges01&request=IBS='))

df_in_aleph["url_ebx"] = df_in_aleph["isbn_ean"].apply(lambda x: f"ebx_link{x}".replace('ebx_link', 'http://aleph.mpg.de/X?op=find&base=ebx01&request=IBN='))


#### Abfragen beim Server


In [37]:
with open('./input/server_responses2', 'w') as fn:  
    for url in df_in_aleph["url_ges"]:
        reply = requests.get(url).text
        a = reply.replace('<?xml version = "1.0" encoding = "UTF-8"?>', '') 
        fn.write(a)

with open('./input/server_responses2', 'r') as f:
    with open('./input/server_responses_transfered2', 'w') as fr:
        for line in f:
            if 'empty' in line:
                fr.write('neu\n')
            elif 'no_records' in line:
                fr.write('vorhanden\n')

df_fwf_ges = pd.read_fwf('./input/server_responses_transfered2', names=["Abfrage_ges"])
df_result2 = df_in_aleph.join(df_fwf_ges)               #df_result2 = pd.concat([df_in_aleph, df_fwf_ges], axis=1)             

In [38]:
#Kontrollabfrage, ob für alle Titel auch Treffer da sind, wird mittelfristig rausfallen
x = df_in_aleph.shape[0]
y = df_fwf_ges.shape[0]
print('Anzahl der eingelesenen Datensätze:', x, '\nAnzahl der Antworten vom Server:   ', y)

Anzahl der eingelesenen Datensätze: 26725 
Anzahl der Antworten vom Server:    26725


In [39]:
df_result2['Abfrage_ges']

0        neu
1        neu
2        neu
3        neu
4        neu
        ... 
26720    neu
26721    neu
26722    neu
26723    neu
26724    neu
Name: Abfrage_ges, Length: 26725, dtype: object

In [40]:
with open('./input/server_responses_ebx2', 'w') as fn:  
    for url in df_result2["url_ebx"]:
        reply = requests.get(url).text
        a = reply.replace('<?xml version = "1.0" encoding = "UTF-8"?>', '') 
        fn.write(a)

with open('./input/server_responses_ebx2', 'r') as f:
    with open('./input/server_responses_transfered_ebx2', 'w') as fr:
        for line in f:
            if 'empty' in line:
                fr.write('neu\n')
            elif 'no_records' in line:
                fr.write('vorhanden\n')

df_fwf_ebx2 = pd.read_fwf('./input/server_responses_transfered_ebx2', names=["Abfrage_ebx"])
df_result_in_aleph = df_result2.join(df_fwf_ebx2)           #df_result_in_aleph = pd.concat([df_result2, df_fwf_ebx2], axis=1)

In [41]:
#Kontrollabfrage, ob für alle Titel auch Treffer da sind
x = df_result_in_aleph.shape[0]
y = df_fwf_ebx2.shape[0]
print('Anzahl der eingelesenen Datensätze:', x, '\nAnzahl der Antworten vom Server:   ', y)

Anzahl der eingelesenen Datensätze: 26725 
Anzahl der Antworten vom Server:    26725


<hr>

## 3. Exportvorbereitungen 


### 1. Neue Titel

In [42]:
df_neu_nicht_einspielen =  df_result_neu.drop(df_result_neu[(df_result_neu["Abfrage_ebx"]== 'neu') & (df_result_neu["Abfrage_ges"] == 'neu')].index)          
# alle Titel rausholen, die in einer der beiden Datenbanken vorhanden waren, diese werden mit den vorhandenen aus Update unten in Excel geschrieben


#### Jetzt Extraktion der Titel zum Einspielen:

In [43]:
df_neu_aleph_einspielen = df_result_neu.loc[((df_result_neu["Abfrage_ebx"]== 'neu') & (df_result_neu["Abfrage_ges"] == 'neu'))]   #das sind die komplett neuen Titel

### 2. In Aleph vorhandene Titel

*Hier ist der Fall: was im Ebooks-Katalog vorhanden ist, muss gelöscht werden*

In [44]:
df_in_aleph_nicht_einspielen = df_result_in_aleph.drop(df_result_in_aleph[(df_result_in_aleph["Abfrage_ebx"] == 'neu') & (df_result_in_aleph["Abfrage_ges"] == 'neu')].index) 
df_in_aleph_nicht_einspielen

,level_0,index,object_id,isbn_ean,title,subtitle,contributor,publisher,series,thesis,...,title_sep,subtitle_sep,subtitle_comparison,subtitle_comparison2,subtitle_all,short_title,url_ges,url_ebx,Abfrage_ges,Abfrage_ebx
536,685,993,11878921,9780415390798,Karl Polanyi and the Paradoxes of the Double M...,,John Vail,Routledge,Routledge Studies in Social and Political Thought,,...,Karl Polanyi and the Paradoxes of the Double M...,,,,,Karl Polanyi and the Paradoxes of the Double M...,http://aleph.mpg.de/X?op=find&base=ges01&reque...,http://aleph.mpg.de/X?op=find&base=ebx01&reque...,vor,neu
586,743,1057,130769734,9780415858762,War as Protection and Punishment,Armed International Intervention at the 'End o...,Teresa Degenhardt,Routledge,Routledge Frontiers of Criminal Justice,,...,War as Protection and Punishment,,Armed International Intervention at the 'End o...,,Armed International Intervention at the 'End o...,War as Protection and Punishment Armed Interna...,http://aleph.mpg.de/X?op=find&base=ges01&reque...,http://aleph.mpg.de/X?op=find&base=ebx01&reque...,vor,neu
665,832,1163,133493308,9781138784925,The Routledge Companion to Environmental Ethics,,Benjamin Hale,Routledge,Routledge Philosophy Companions,,...,The Routledge Companion to Environmental Ethics,,,,,The Routledge Companion to Environmental Ethic...,http://aleph.mpg.de/X?op=find&base=ges01&reque...,http://aleph.mpg.de/X?op=find&base=ebx01&reque...,vor,neu
804,988,1357,137861669,9781138293823,"Law, Space, and the Vehicular Environment",Pavement and Asphalt,Sarah Marusek,Routledge,"Space, Materiality and the Normative",,...,"Law, Space, and the Vehicular Environment",,Pavement and Asphalt,,Pavement and Asphalt,"Law, Space, and the Vehicular Environment Pave...",http://aleph.mpg.de/X?op=find&base=ges01&reque...,http://aleph.mpg.de/X?op=find&base=ebx01&reque...,vor,neu
1011,1217,1643,144553561,9781138491946,The Diaspora's Role in Africa,"Transculturalism, Challenges, and Development",Stella-Monica N. Mpande,Routledge,Rethinking Development,,...,The Diaspora's Role in Africa,,"Transculturalism, Challenges, and Development",,"Transculturalism, Challenges, and Development",The Diaspora's Role in Africa Transculturalism...,http://aleph.mpg.de/X?op=find&base=ges01&reque...,http://aleph.mpg.de/X?op=find&base=ebx01&reque...,vor,neu
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26512,31793,4782,157121198,9780367681524,Aid and Influence,"Patronage, Power and Politics",Stephen Browne,Routledge,,,...,Aid and Influence,,"Patronage, Power and Politics",,"Patronage, Power and Politics","Aid and Influence Patronage, Power and Politic...",http://aleph.mpg.de/X?op=find&base=ges01&reque...,http://aleph.mpg.de/X?op=find&base=ebx01&reque...,vor,neu
26544,31831,1559,143382178,9781138671324,African Americans and the Mississippi River,"Race, History, and the Environment",Dorothy Zeisler-Vralsted,Routledge,Routledge Environmental Humanities,,...,African Americans and the Mississippi River,,"Race, History, and the Environment",,"Race, History, and the Environment",African Americans and the Mississippi River Ra...,http://aleph.mpg.de/X?op=find&base=ges01&reque...,http://aleph.mpg.de/X?op=find&base=ebx01&reque...,vor,neu
26562,31852,20020,163737954,9780367648596,Advancing Culturally Responsive Research and R...,"Qualitative, Quantitative, and Mixed Methods",Penny A. Pasque,Routledge,,,...,Advancing Culturally Responsive Research and R...,,"Qualitative, Quantitative, and Mixed Methods",,"Qualitative, Quantitative, and Mixed Methods",Advancing Culturally Responsive Research and R...,http://aleph.mpg.de/X?op=find&base=ges01&reque...,http://aleph.mpg.de/X?op=find&base=ebx01&reque...,vor,neu
26620,31918,29345,165866839,9780367419905,Abolish Criminology,,Viviane Saleh-Hanna,Routledge,Routledge Studies in Penal Abolition and Trans...,,...,Abolish Criminology,,,,,Abolish Criminology / Viviane Saleh-Hanna,http://aleph.mpg.de/X?op=find&base=ges01&reque...,http://aleph.mp

In [45]:
# Erzeugen der Datei zum Löschen der Titel

with open("./output/ges02_weg", "a", encoding="utf-8") as fa:  #durch das Encoding hier, kommen Sonderzeichen richtig rüber
    for i in df_in_aleph_nicht_einspielen.index:
        fa.write(df_in_aleph_nicht_einspielen["ids"][i]+'GES02'+'\n')

Für die Statistik und Kontrolle zusammenführen aller Titel, die nicht eingespielt werden und Ausgabe in einer CSV

In [46]:
df_gesamt_nicht_einspielen = df_in_aleph_nicht_einspielen.append(df_neu_nicht_einspielen)   

In [47]:
date = time.strftime("%Y_%m_%d")                                              # Zeit erfassen für Dateibenennung

df_gesamt_nicht_einspielen["object_id"] = df_gesamt_nicht_einspielen.object_id.astype(str)  # wandelt die spalte von Int64 zu Object um, so dass es in Excel korrekt eingelesen wird
df_gesamt_nicht_einspielen["isbn_ean"] = df_gesamt_nicht_einspielen.isbn_ean.astype(str)
df_gesamt_nicht_einspielen = df_gesamt_nicht_einspielen.drop(columns=["url_ebx", "url_ges", "cover", "title_sep", "subtitle_comparison", "subtitle_comparison2", "subtitle_all", "subtitle_sep"]) # unnötige Spalten entfernen

df_neu_nicht_einspielen.to_csv('./output/Vorhandene_Titel_'+date+'.csv')   

#### Jetzt Extraktion der Titel zum Updaten:

In [48]:
df_in_aleph_update = df_result_in_aleph.loc[(df_result_in_aleph["Abfrage_ebx"]== 'neu') & (df_result_in_aleph["Abfrage_ges"] == 'neu')] # prüfen, ob es wirklich ein Datenupdate gab, sonst nicht neu einspielen??

<hr>

#### Für die Logdatei Ermittlung verschiedener Zahlen und hier zur direkten Ansicht ausgegeben

In [49]:
#Kontrollmechanismus, ob für alle Titel auch Treffer da sind
x = df.shape[0]
a = df_ohne_dubletten.shape[0]
c = df_dubletten.shape[0]
b = df_dubl_einspielen.shape[0] #Auswahl der neuen Treffer
m = df_in_aleph_nicht_einspielen.shape[0]
n = df_ganz_neu.shape[0]
h = df_neu_aleph_einspielen.shape[0]
j = df_neu_nicht_einspielen.shape[0]
o = df_in_aleph_update.shape[0]
g = df_in_aleph.shape[0]
k = df_alephIDs.shape[0]
z = df_neu_aleph_einspielen.shape[0]       #neu ermitteln aus neuen
l = df_aleph_loeschen.shape[0]
y = df_dubl_nicht_einspielen.shape[0]


print('Kleine Statistik:\n=====================================',
    '\nGelieferte Datensätze:             ', x,
    '\n--------------------------------------------', 
    '\nSätze ohne Dubletten               ', a,
    '\n   Dubletten:              ', c,
    '\n   Auswahl zum Einspielen  ', b,
    '\n   in Aleph löschen        ', y,
    '\nZu prüfende Datensätze:            ', a+b,
    '\n--------------------------------------------', 
    '\nPrüfung Titel in Aleph:            ', g,
    '\n   Davon als Update        ', o,
    '\n   Davon in Aleph löschen  ', m ,    
    '\nPrüfung neue Titel    :            ', n,
    '\n   Davon ganz neu          ', h,
    '\n   Davon bereits Bestand   ', j,      
    '\n--------------------------------------------', 
    '\nTitel, die in Aleph verarbeitet werden: ', o+h,
    '\n--------------------------------------------', 
    '\nDatensätze GES02 vor Einspielen:   ', k,
    '\nNicht mehr im Export               -', l,
    '\nTitel in Aleph löschen, da Bestand -', m,
    '\nZu löschen, da Dublette            -', y,
    '\nNeue Titel für Aleph               +', h,
    '\n--------------------------------------------', 
    '\nIn Aleph nach Einspielen:          ', k-l-m-y+h,)

#hier entsprechende Einträge für die Log-Datei

with open ('./log/pda_import_log.txt', 'a') as log:
    log.write("\nGelieferte Datensätze:             " + str(x))
    log.write("\n--------------------------------------------\n")
    log.write("\nSätze ohne Dubletten               " + str(a))
    log.write("\n   Dubletten:              " + str(c))
    log.write("\n   Davon in Aleph löeschen " + str(y))
    log.write("\n   Auswahl zum Einspielen  " + str(b))
    log.write("\nZu prüfende Datensätze:            " + str(a+b))
    log.write("\n--------------------------------------------\n")
    log.write("\nPrüfung Titel in Aleph:            " + str(g))
    log.write("\n   Davon als Update        " + str(o))
    log.write("\n   Davon in Aleph löschen  " + str(m) + "    \n")   
    log.write("\nPrüfung neue Titel    :            " + str(n))
    log.write("\n   Davon ganz neu          " + str(h))
    log.write("\n   Davon bereits Bestand   " + str(j) + "    \n")     
    log.write("\n--------------------------------------------\n")
    log.write("\nTitel, die in Aleph verarbeitet werden: " + str(o+h))
    log.write("\n--------------------------------------------\n")
    log.write("\nDatensätze GES02 vor Einspielen:   " + str(k))
    log.write("\nNicht mehr im Export               -" + str(l))
    log.write("\nZu löschender dubletter Titel      -" + str(y))
    log.write("\nTitel in Aleph löschen, da Bestand -" + str(m))
    log.write("\nNeue Titel für Aleph               +" + str(h))
    log.write("\n--------------------------------------------\n")
    log.write("\nIn Aleph nach Einspielen:          " + str(k-l-m-y+h))
    log.write("\n--------------------------------------------\n")

Kleine Statistik:
Gelieferte Datensätze:              38142 
-------------------------------------------- 
Sätze ohne Dubletten                26080 
   Dubletten:               12062 
   Auswahl zum Einspielen   5974 
   in Aleph löschen         0 
Zu prüfende Datensätze:             32054 
-------------------------------------------- 
Prüfung Titel in Aleph:             26725 
   Davon als Update         26185 
   Davon in Aleph löschen   540 
Prüfung neue Titel    :             5329 
   Davon ganz neu           3098 
   Davon bereits Bestand    2231 
-------------------------------------------- 
Titel, die in Aleph verarbeitet werden:  29283 
-------------------------------------------- 
Datensätze GES02 vor Einspielen:    31360 
Nicht mehr im Export               - 3 
Titel in Aleph löschen, da Bestand - 540 
Zu löschen, da Dublette            - 0 
Neue Titel für Aleph               + 3098 
-------------------------------------------- 
In Aleph nach Einspielen:           33915


<hr>

## 4. Exportdateien Aufbereiten

#### Zielformat für das Einspielen in Aleph:

    000000001 LDR   L -----nM2.01200024------h              
    000000001 020   L $$a (object_id))
    000000001 030   L $$aaz||rrrza||||
    000000001 051   L $$am|||||||
    000000001 070   L $$aSchweitzer
    000000001 077   L $$aMonographie
    000000001 078   L $$aSchweitzer
    000000001 082   L $$azum Bestellen
    000000001 100   L $$a (contributor_1)
    000000001 104   L $$a (contributor_2)
    000000001 108   L $$a (contributor_3)
    000000001 331   L $$a (title_sep)
    000000001 335   L $$a (subtitle_all)
    000000001 403   L $$a (edition_number / edition_text)  #noch prüfen, was besser zu verwenden ist 
    000000001 419   L $$b (publisher) $$a (date_combined)
    000000001 433   L $$a (pages)
    000000001 451   L $$a (series)
    000000001 520   L $$a (thesis)
    000000001 540   L $$a (isbn_ean)
    000000001 656   L $$a (cover)
    000000001 750   L $$a (description)
    000000001 655   L $$zOrder me$$umailto:bib@mpifg.de?subject=Bestellwunsch        
    
Anmerkung zum Feld 655: die URL wird NACH dem Einspielen in Aleph mit der Datensatz-ID angereichert (siehe Juypter-Notebook "Link-Anreicherung"), um einen klaren Bestellink für den Kaufvorschlag zu haben

*Hierfür werden immer die Feldbenennung und bestimmte Codierungen VOR den Inhalt - in Klammern de Bezeichnung der entsprechenden Spalte - gesetzt, bzw. erfoderliche Felder komplett neu hinzugefügt.   
Am Anfang jeder Zeile braucht Aleph eine 9-Stellige eindeutige Zahl pro Titel.*   

*Manchmal ließ sich der Inhalt einer Spalte direkt in die Datei schreiben, manchmal musst die Spalte zuvor über apply aufbereitet werden.* 

## 1. Aufbereiten der neuen Titel

In [50]:
df_aleph_einspielen = df_neu_aleph_einspielen           #zur vereinfachten Wiederverwertung des alten Codes

In [64]:
df_aleph_einspielen["020"] = df_aleph_einspielen["object_id"].apply(lambda x: f"020   L $$a{x}") 
del df_aleph_einspielen["object_id"]                                                                      #um das df nicht unnötig anwachsen zu lassen, jeweils alte Spalte löschen

#### Besondere Aufbereitung der Personendaten

*Da bis zu 3 Personen in einer Spalte zu finden sind, werden diese im Discovery nicht getrennt suchbar, darum werden sie gesplittet. Für die Dublettenkontrolle hat sich das als irrelevant erwiesen, darum erfolgt dieser Schritt erst hier.*

In [65]:
person = df_aleph_einspielen["contributor"].str.split(';', expand=True)                         #Für saubere Daten, die Autorenangabe splitten und in getrennte Felder schreiben

df_aleph_einspielen["contributor_1"]= person[0]
df_aleph_einspielen["contributor_2"]= person[1]
df_aleph_einspielen["contributor_3"]= person[2]

df_aleph_einspielen["contributor_1"]= df_aleph_einspielen["contributor_1"].replace(np.nan, '', regex=True)
df_aleph_einspielen["contributor_2"]= df_aleph_einspielen["contributor_2"].replace(np.nan, '', regex=True)
df_aleph_einspielen["contributor_3"]= df_aleph_einspielen["contributor_3"].replace(np.nan, '', regex=True)


#### Besondere Aufbereitung des Erscheinungsdatum und Erscheinungsjahres

*In der Auswahl unserer Titel befinden sich auch im Erscheinen befindliche Titel der kommenden Monate. Diese Information möchten wir gerne im Discovery sichtbar machen. Hierfür bleibt uns nur Aleph-Feld 419c, das dem Erscheinungsjahr vorbehalten ist.   
Wunsch ist es: Wenn des Erscheinungsdatum weiter als 10 Tage weg vom heutigen Datum ist, soll das komplette Datum angezeigt werden, ansonsten nur das Erscheinungsjahr.*

Zur Umsetzung muss die Spalte "publication_date" in ein Datum verwandelt werden und nach den genannten Kriterien unterschiedlich angezeigt

In [166]:
df_aleph_einspielen["publ_data_repl"]= df_aleph_einspielen["publication_date"].astype(str).str.replace('00','01') # korrektur komischer Daten
#df_aleph_einspielen["publ_data_repl"]

In [167]:
df_aleph_einspielen["publ_date_date"] = pd.to_datetime(df_aleph_einspielen["publ_data_repl"].astype(str), format='%Y%m%d', errors='coerce') #Umwandlung in Datum (Zahl geht nicht)
#df_aleph_einspielen["publ_date_date"]

In [ ]:
today = pd.Timestamp(datetime.today().date())  # Heute 

def transform_date(date):
    if date > today:
        return date.strftime('%Y-%m-%d')
    else:
        return date.year

# Apply the function to create the new column
df_aleph_einspielen['date_combined'] = df_aleph_einspielen['publ_date_date'].apply(transform_date)
#df_aleph_einspielen['date_combined']

0        2025-12-31
1              2022
2              2022
3        2026-06-01
4              2022
            ...    
26262          2022
26263    2025-03-20
26264          2025
26265          2024
26266    2025-04-01
Name: date_combined, Length: 26267, dtype: object

In [170]:
df_aleph_einspielen['isbn_ean'] = df_aleph_einspielen['isbn_ean'].astype(np.int64)   #da die Zahl als Object genommen wurde, bekam sie ein .0 angehängt, das ist durch umwandeln in Zahl weg

df_aleph_einspielen['isbn_ean']

0        9783406582387
1        9783406753732
2        9783406767890
3        9783848728732
4        9783170314221
             ...      
26262    9783030875183
26263    9781350242401
26264    9781009523103
26265    9781538180631
26266    9781961856783
Name: isbn_ean, Length: 26267, dtype: int64

In [171]:
df_aleph_einspielen["419b"] = df_aleph_einspielen["publisher"].apply(lambda x: f"419   L $$b{x}") 
df_aleph_einspielen["419c"] = df_aleph_einspielen["date_combined"].apply(lambda x: f"$$c{x}")                  

df_aleph_einspielen["419"] = df_aleph_einspielen["419b"]+df_aleph_einspielen["419c"]                           #Für die Korrekte Eingabe brauche ich Verlag und Jahr in einer Spalte

In [172]:
df_aleph_einspielen["403"] = df_aleph_einspielen["edition_text"].apply(lambda x: f"403   L $$a{x}") 
df_aleph_einspielen["433"] = df_aleph_einspielen["pages"].apply(lambda x: f"433   L $$b{x}")
df_aleph_einspielen["451"] = df_aleph_einspielen["series"].apply(lambda x: f"451   L $$b{x}") 
df_aleph_einspielen["520"] = df_aleph_einspielen["thesis"].apply(lambda x: f"520   L $$a{x}") 
df_aleph_einspielen["540"] = df_aleph_einspielen["isbn_ean"].apply(lambda x: f"540   L $$a{x}") 
df_aleph_einspielen["656"] = df_aleph_einspielen["cover"].apply(lambda x: f"656   L $$u{x}") 
df_aleph_einspielen["750"] = df_aleph_einspielen["description"].apply(lambda x: f"750   L $$a{x}") 

#### Für das Durchzählen der Titel braucht es eine neue Spalte

In [173]:
# hier entsteht eine neue Spalte mit Zahlen ab 1 durchgehend gezählt, die für den korrekten Import der Daten in Aleph nötig ist
x = df_aleph_einspielen.shape[0]   
df_aleph_einspielen["id"] = range(1,x+1)                                                       #Notwendig ist die Zählung ab 1, da Aleph sonst nicht korrekt einließt
df_aleph_einspielen["id"] = df_aleph_einspielen["id"].apply(lambda x: f"{x:09d}")              #Die Zahl muss 9-Stellig aufgefüllt werden



#### Vorbereitungen abgeschlossen, jetzt das Schreiben der Datei im Aleph-Sequential-Format:

In [174]:
with open("./output/ges02_neu", "w", encoding="utf-8") as fa:  #durch das Encoding hier, kommen Sonderzeichen richtig rüber
    for i in df_aleph_einspielen.index:
        fa.write(df_aleph_einspielen["id"][i]+' LDR   L -----nM2.01200024------h'+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' '+df_aleph_einspielen["020"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 030   L $$aaz||rrrza||||'+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 051   L $$am|||||||m|||||||'+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 077   L $$aMonographie'+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 078   L $$aSchweitzer'+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 082   L $$azum Bestellen'+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 100   L $$a'+df_aleph_einspielen["contributor_1"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 104   L $$a'+df_aleph_einspielen["contributor_2"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 108   L $$a'+df_aleph_einspielen["contributor_3"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 331   L $$a'+df_aleph_einspielen["title_sep"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 335   L $$a'+df_aleph_einspielen["subtitle_all"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' '+df_aleph_einspielen["403"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' '+df_aleph_einspielen["419"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' '+df_aleph_einspielen["433"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' '+df_aleph_einspielen["451"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' '+df_aleph_einspielen["520"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' '+df_aleph_einspielen["540"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 655   L $$zOrder me$$umailto:bib@mpifg.de?subject=Bestellwunsch'+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' '+df_aleph_einspielen["656"][i]+'$$3Cover\n')
        fa.write(df_aleph_einspielen["id"][i]+' '+df_aleph_einspielen["750"][i]+'\n')

<hr>

### 2. Daten für Update

*es wird anhand der Spalte "last_modified" geprüft, ob die Titel seit dem letzten Einspielen, bzw. seit 30 Tagen ein Update erfahren haben*


Dies mache ich, um die Menge zum Updaten zu reduzieren. Es werden nur die rausgezogen, die tatsächlich ein neueres Aktualisierungsdatum haben.

In [191]:
one_month_ago = pd.Timestamp(one_month_ago)
one_month_ago

Timestamp('2025-02-11 17:29:59.012199')

In [192]:
df_in_aleph_update['last_modified'] = pd.to_datetime(df_in_aleph_update.last_modified)          #Umwandlung, da Spalteninhalt object ist
df_in_aleph_update['last_modified']

0       2025-03-09 21:36:50
1       2025-02-07 06:11:51
2       2025-02-11 05:29:42
3       2025-02-18 06:40:09
4       2025-02-20 20:40:50
                ...        
27679   2025-02-23 20:07:03
27680   2025-02-28 17:30:14
27681   2025-02-26 17:28:55
27682   2025-02-12 02:13:37
27683   2025-02-27 17:56:55
Name: last_modified, Length: 27684, dtype: datetime64[ns]

In [ ]:
df_in_aleph_update_true = df_in_aleph_update[df_in_aleph_update['last_modified'].dt.to_period('M') == one_month_ago.to_period('M')]

#df_in_aleph_update_true

,level_0,index,object_id,isbn_ean,title,subtitle,contributor,publisher,series,thesis,...,Field,L,Content,LEN,title_sep,subtitle_sep,subtitle_comparison,subtitle_comparison2,subtitle_all,short_title
1,1,1,2522807,9783848768073,Klimaschutzrecht,Bundes-Klimaschutzgesetz | Landesklimagesetze ...,,Nomos,NomosHandkommentar,,...,020,L,$$a2522807,10.0,Klimaschutzrecht,,Bundes-Klimaschutzgesetz | Landesklimagesetze ...,,Bundes-Klimaschutzgesetz | Landesklimagesetze ...,Klimaschutzrecht Bundes-Klimaschutzgesetz | La...
2,2,2,2308946,9783848728497,Rehabilitation und Teilhabe,,,Nomos,Kompendien der Sozialen Arbeit Band 5,,...,020,L,$$a2308946,10.0,Rehabilitation und Teilhabe,,,,,Rehabilitation und Teilhabe /
3,3,3,2489418,9783825254476,Biodiversität? Frag doch einfach!,Klare Antworten aus erster Hand,Heike Feldhaar,UTB,Frag doch einfach!,,...,020,L,$$a2489418,10.0,Biodiversität? Frag doch einfach!,,Klare Antworten aus erster Hand,,Klare Antworten aus erster Hand,Biodiversität? Frag doch einfach! Klare Antwor...
4,4,4,2220214,9783406701948,Besteuerung privater Kapitalanlagen,"Finanzinstrumente, Investmentanteile, Immobili...",Hans-Jürgen A. Feyerabend,C.H.BECK,,,...,020,L,$$a2220214,10.0,Besteuerung privater Kapitalanlagen,,"Finanzinstrumente, Investmentanteile, Immobili...",,"Finanzinstrumente, Investmentanteile, Immobili...",Besteuerung privater Kapitalanlagen Finanzinst...
5,5,5,218910,9783406645877,Grundkurs Steuerrecht,,Rainer Wernsmann;Christian Thiemann,C.H.BECK,Grundkurse,,...,020,L,$$a218910,9.0,Grundkurs Steuerrecht,,,,,Grundkurs Steuerrecht / Rainer Wernsmann;Chri...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27679,53939,7266,158629965,9783110710045,(Re-)Mobilizing Voters in Britain and the Unit...,Political Strategies from Parties and Grassroo...,Gregory Benedetti,De Gruyter Oldenbourg,,,...,020,L,$$a158629965,12.0,(Re-)Mobilizing Voters in Britain and the Unit...,,Political Strategies from Parties and Grassroo...,,Political Strategies from Parties and Grassroo...,(Re-)Mobilizing Voters in Britain and the Unit...
27680,53941,54779,169951393,9781032411460,(Co)Designing Hope,Aqueous Landscapes in Transition,Laura Cipriani,Routledge,,,...,020,L,$$a169951393,12.0,(Co)Designing Hope,,Aqueous Landscapes in Transition,,Aqueous Landscapes in Transition,(Co)Designing Hope Aqueous Landscapes in Trans...
27681,53945,51527,169483493,9781032520728,'Just Like Us'?: The Politics of Ministerial P...,The Politics of Ministerial Promotion in UK Go...,Bill Jones,Routledge,Routledge Studies in British Politics,,...,020,L,$$a169483493,12.0,'Just Like Us'?,The Politics of Ministerial Promotion in UK G...,The Politics of Ministerial Promotion in UK Go...,,The Politics of Ministerial Promotion in UK Go...,'Just Like Us'? The Politics of Ministerial Pr...
27682,53946,20200,163496075,9783030962708,'Going Native?',Settler Colonialism and Food,Ronald Ranta,Palgrave Macmillan,Food and Identity in a Globalising World,,...,020,L,$$a163496075,12.0,'Going Native?',,Settler Colonialism and Food,,Settler Colonialism and Food,'Going Native?' Settler Colonialism and Food /...


#### Erstellen der Update-Export-Datei

In [196]:
df_in_aleph_update_true["020"] = df_in_aleph_update_true["object_id"].apply(lambda x: f"020   L $$a{x}") 
del df_in_aleph_update_true["object_id"]                                                                      #um das df nicht unnötig anwachsen zu lassen, jeweils alte Spalte löschen

In [197]:
#Aufbereitung Personendaten

person = df_in_aleph_update_true["contributor"].str.split(';', expand=True)                         #Für saubere Daten, die Autorenangabe splitten und in getrennte Felder schreiben

df_in_aleph_update_true["contributor_1"]= person[0]
df_in_aleph_update_true["contributor_2"]= person[1]
df_in_aleph_update_true["contributor_3"]= person[2]

df_in_aleph_update_true["contributor_1"]= df_in_aleph_update_true["contributor_1"].replace(np.nan, '', regex=True)
df_in_aleph_update_true["contributor_2"]= df_in_aleph_update_true["contributor_2"].replace(np.nan, '', regex=True)
df_in_aleph_update_true["contributor_3"]= df_in_aleph_update_true["contributor_3"].replace(np.nan, '', regex=True)


### Neue Schritte zur Aufbereitung des Erscheinungsdatums, da der Code von oben hier nicht ging - warum auch immer

In [203]:
df_in_aleph_update_true["publ_data_repl"]= df_in_aleph_update_true["publication_date"].astype(str).str.replace('00','01')
df_in_aleph_update_true["publ_date_date"] = pd.to_datetime(df_in_aleph_update_true["publ_data_repl"].astype(str), format='%Y%m%d', errors='coerce')

# Apply the function von oben to create the new column
df_in_aleph_update_true['date_combined'] = df_in_aleph_update_true['publ_date_date'].apply(transform_date)
df_in_aleph_update_true['date_combined']

1        2025-05-01
2        2025-07-01
3        2025-05-12
4        2025-08-31
5        2025-06-30
            ...    
27679          2024
27680          2024
27681          2024
27682          2023
27683          2023
Name: date_combined, Length: 15411, dtype: object

In [204]:
df_in_aleph_update_true['date_combined'].value_counts()

2024          8445
2023          6741
2025            70
2025-06-01      14
2025-12-01      13
              ... 
2025-12-02       1
2025-05-22       1
2025-10-15       1
2025-10-19       1
2026-04-30       1
Name: date_combined, Length: 71, dtype: int64

In [205]:
df_in_aleph_update_true['isbn_ean'] = df_in_aleph_update_true['isbn_ean'].astype(np.int64)   #da die Zahl als Object genommen wurde, bekam sie ein .0 angehängt, das ist durch umwandeln in Zahl weg

df_in_aleph_update_true['isbn_ean']

1        9783848768073
2        9783848728497
3        9783825254476
4        9783406701948
5        9783406645877
             ...      
27679    9783110710045
27680    9781032411460
27681    9781032520728
27682    9783030962708
27683    9781350276765
Name: isbn_ean, Length: 15411, dtype: int64

In [206]:
# Bearbeitung der Felder

df_in_aleph_update_true["419b"] = df_in_aleph_update_true["publisher"].apply(lambda x: f"419   L $$b{x}") 
df_in_aleph_update_true["419c"] = df_in_aleph_update_true["date_combined"].apply(lambda x: f"$$c{x}")                  

df_in_aleph_update_true["419"] = df_in_aleph_update_true["419b"]+df_in_aleph_update_true["419c"]                           #Für die Korrekte Eingabe brauche ich Verlag und Jahr in einer Spalte

df_in_aleph_update_true["403"] = df_in_aleph_update_true["edition_text"].apply(lambda x: f"403   L $$a{x}") 
df_in_aleph_update_true["433"] = df_in_aleph_update_true["pages"].apply(lambda x: f"433   L $$b{x}")
df_in_aleph_update_true["451"] = df_in_aleph_update_true["series"].apply(lambda x: f"451   L $$b{x}") 
df_in_aleph_update_true["520"] = df_in_aleph_update_true["thesis"].apply(lambda x: f"520   L $$a{x}") 
df_in_aleph_update_true["540"] = df_in_aleph_update_true["isbn_ean"].apply(lambda x: f"540   L $$a{x}") 
df_in_aleph_update_true["656"] = df_in_aleph_update_true["cover"].apply(lambda x: f"656   L $$u{x}") 
df_in_aleph_update_true["750"] = df_in_aleph_update_true["description"].apply(lambda x: f"750   L $$a{x}") 

In [207]:
# Schreiben der Ausgabedatei, hier kleine Unterschiede zu den neuen Titeln. Vorhandene ids und bestimmte Felder können nicht verändert sein, brauchen also nicht übernommen zu werden.

with open("./output/ges02_update", "w", encoding="utf-8") as fa:  #durch das Encoding hier, kommen Sonderzeichen richtig rüber
    for i in df_in_aleph_update_true.index:
        fa.write(df_in_aleph_update_true["ids"][i]+' 100   L $$a'+df_in_aleph_update_true["contributor_1"][i]+'\n')
        fa.write(df_in_aleph_update_true["ids"][i]+' 104   L $$a'+df_in_aleph_update_true["contributor_2"][i]+'\n')
        fa.write(df_in_aleph_update_true["ids"][i]+' 108   L $$a'+df_in_aleph_update_true["contributor_3"][i]+'\n')
        fa.write(df_in_aleph_update_true["ids"][i]+' 331   L $$a'+df_in_aleph_update_true["title_sep"][i]+'\n')
        fa.write(df_in_aleph_update_true["ids"][i]+' 335   L $$a'+df_in_aleph_update_true["subtitle_all"][i]+'\n')
        fa.write(df_in_aleph_update_true["ids"][i]+' '+df_in_aleph_update_true["403"][i]+'\n')
        fa.write(df_in_aleph_update_true["ids"][i]+' '+df_in_aleph_update_true["419"][i]+'\n')
        fa.write(df_in_aleph_update_true["ids"][i]+' '+df_in_aleph_update_true["433"][i]+'\n')
        fa.write(df_in_aleph_update_true["ids"][i]+' '+df_in_aleph_update_true["451"][i]+'\n')
        fa.write(df_in_aleph_update_true["ids"][i]+' '+df_in_aleph_update_true["520"][i]+'\n')
        fa.write(df_in_aleph_update_true["ids"][i]+' '+df_in_aleph_update_true["540"][i]+'\n')
        fa.write(df_in_aleph_update_true["ids"][i]+' '+df_in_aleph_update_true["656"][i]+'$$3Cover\n')
        fa.write(df_in_aleph_update_true["ids"][i]+' '+df_in_aleph_update_true["750"][i]+'\n')

<hr>

## Abschließende Dinge

In [86]:
#Abschließender Eintrag in Log-Datei
endtime = time.strftime('%H:%M')

with open ('./log/pda_import_log.txt', 'a') as log:                                                  # Da diese Log-Datei nicht unmittelbar gebraucht wird, hier fortlaufendes Schreiben in eine Datei
    log.write('\n                                     beendet ')
    log.write(endtime)
    log.write("\n============================================================\n\n")

Kopien bestimmter Daten zur Einsicht bzw. für Prüfzwecke

In [87]:
df_aleph_einspielen.to_csv('./output/Eingespielte_Titel_'+date+'.csv') 
df_in_aleph_update_true.to_csv('./output/Update_Titel_'+date+'.csv')

df_in_aleph_nicht_einspielen.to_csv('./output/Aleph_loeschen_ebx_vorh_'+date+'.csv')

<hr>

Folgende Schritte müssen mit den Dateien ausgeführt werden: 


1. Einspielen der Datei pda_ges01 als neue Titel in Aleph, hier dann auch Export der urls und Anreicherung mit der Aleph-ID mittels "mailto_link_skript.ipynb"
2. Einspielen der DAtei pda_update als "Änderungen bestehender Datensätze in Aleph"
3. Einspielen und löschen der Titel die in ges02_loeschen_1 und ges02_loeschen_2 vorhanden sind